In [4]:
# =========================
# 0) Install deps (Kaggle / Colab)
# =========================
# If you're on Kaggle, you can run this in a notebook cell.
!pip -q install opencv-python-headless scikit-learn mlxtend tqdm scikit-image kagglehub



In [ ]:
# =========================
# 1) Imports + config
# =========================
import os, glob, random
import numpy as np
import cv2
from tqdm import tqdm

from sklearn.cluster import MiniBatchKMeans
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

from skimage.feature import local_binary_pattern


/Users/dylesm/Frugal/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# =========================
# 2) Point this at your Kaggle dataset images
# =========================
# Change this to your dataset root.


# Download latest version
# path = kagglehub.dataset_download("vesuvius13/formula-one-cars")
path = "/Users/dylesm/.cache/kagglehub/datasets/vesuvius13/formula-one-cars/versions/1"

print("Path to dataset files:", path)

DATA_ROOT = path

# Common image extensions
EXTS = (".jpg", ".jpeg", ".png", ".bmp", ".webp")

def find_images(root):
    paths = []
    for dirpath, _, filenames in os.walk(root):
        for fn in filenames:
            if fn.lower().endswith(EXTS):
                paths.append(os.path.join(dirpath, fn))
    return paths

image_paths = find_images(DATA_ROOT)
print("Found images:", len(image_paths))
print("Example:", image_paths[:3])


Path to dataset files: /Users/dylesm/.cache/kagglehub/datasets/vesuvius13/formula-one-cars/versions/1
Found images: 2409
Example: ['/Users/dylesm/.cache/kagglehub/datasets/vesuvius13/formula-one-cars/versions/1/Formula One Cars/Williams F1 car/00000400.jpg', '/Users/dylesm/.cache/kagglehub/datasets/vesuvius13/formula-one-cars/versions/1/Formula One Cars/Williams F1 car/00000372.jpg', '/Users/dylesm/.cache/kagglehub/datasets/vesuvius13/formula-one-cars/versions/1/Formula One Cars/Williams F1 car/00000414.jpg']


In [9]:
# =========================
# 3) Load + normalize images (resize)
# =========================
IMG_SIZE = (256, 256)   # Keep it manageable to start
MAX_IMAGES = 300        # Start small; scale later

random.shuffle(image_paths)
image_paths_small = image_paths[:min(MAX_IMAGES, len(image_paths))]

def load_image_bgr(path, size=IMG_SIZE):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    if img is None:
        return None
    img = cv2.resize(img, size, interpolation=cv2.INTER_AREA)
    return img

imgs = []
kept_paths = []
for p in tqdm(image_paths_small):
    im = load_image_bgr(p)
    if im is not None:
        imgs.append(im)
        kept_paths.append(p)

print("Loaded:", len(imgs))


100%|██████████| 300/300 [00:05<00:00, 52.04it/s]

Loaded: 300


In [10]:
# =========================
# 4) Split into patches (per image)
# =========================
PATCH = 32  # 32x32 patches
# With IMG_SIZE=256 and PATCH=32 => 8x8 grid

def image_to_patches(img_bgr, patch=PATCH):
    h, w = img_bgr.shape[:2]
    assert h % patch == 0 and w % patch == 0
    patches = []
    coords = []
    for y in range(0, h, patch):
        for x in range(0, w, patch):
            patches.append(img_bgr[y:y+patch, x:x+patch])
            coords.append((y // patch, x // patch))  # (row, col) in patch grid
    return patches, coords, (h // patch, w // patch)

# quick test
patches0, coords0, grid0 = image_to_patches(imgs[0])
print("Patches:", len(patches0), "Grid:", grid0)


Patches: 64 Grid: (8, 8)


In [11]:
# =========================
# 5) Patch feature extractor (simple + robust)
#    - Color histogram in HSV
#    - Texture via LBP histogram
# =========================
LBP_P = 8
LBP_R = 1
LBP_METHOD = "uniform"
LBP_BINS = LBP_P + 2  # uniform LBP bins

def hsv_hist(patch_bgr, h_bins=8, s_bins=8, v_bins=8):
    hsv = cv2.cvtColor(patch_bgr, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0,1,2], None, [h_bins,s_bins,v_bins], [0,180, 0,256, 0,256])
    hist = hist.flatten().astype(np.float32)
    hist /= (hist.sum() + 1e-8)
    return hist

def lbp_hist(patch_bgr):
    gray = cv2.cvtColor(patch_bgr, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray, P=LBP_P, R=LBP_R, method=LBP_METHOD)
    # histogram over LBP codes
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, LBP_BINS+1), range=(0, LBP_BINS))
    hist = hist.astype(np.float32)
    hist /= (hist.sum() + 1e-8)
    return hist

def patch_features(patch_bgr):
    return np.concatenate([hsv_hist(patch_bgr), lbp_hist(patch_bgr)], axis=0)

# feature length check
feat0 = patch_features(patches0[0])
print("Feature dim:", feat0.shape)


Feature dim: (522,)


In [12]:
# =========================
# 6) Build a big feature matrix from many patches (for clustering)
# =========================
# To keep memory sane, subsample patches per image at first.
PATCHES_PER_IMAGE = 64  # take all if image is 8x8=64 patches; otherwise sample

all_feats = []
all_meta = []  # (img_idx, patch_row, patch_col) for later mapping

for i, img in enumerate(tqdm(imgs)):
    patches, coords, (gh, gw) = image_to_patches(img)
    idxs = list(range(len(patches)))
    if len(idxs) > PATCHES_PER_IMAGE:
        idxs = random.sample(idxs, PATCHES_PER_IMAGE)

    for j in idxs:
        all_feats.append(patch_features(patches[j]))
        pr, pc = coords[j]
        all_meta.append((i, pr, pc))

X = np.vstack(all_feats)
print("Total patch samples:", X.shape)


100%|██████████| 300/300 [00:01<00:00, 171.64it/s]

Total patch samples: (19200, 522)


In [ ]:
# =========================
# 7) Quantize patches into "visual words" with k-means
# =========================
K = 200  # vocabulary size (start 100-300)
kmeans = MiniBatchKMeans(n_clusters=K, batch_size=4096, random_state=0)
kmeans.fit(X)

print("Trained kmeans with K =", K)


In [16]:
# =========================
# 8) Assign every patch in every image to a visual word
# =========================
def image_words_grid(img_bgr, patch=PATCH):
    patches, coords, (gh, gw) = image_to_patches(img_bgr, patch=patch)
    feats = np.vstack([patch_features(p) for p in patches])
    words = kmeans.predict(feats)  # [num_patches]
    grid = words.reshape(gh, gw)
    return grid  # shape: (gh, gw)

word_grids = []
for img in tqdm(imgs):
    word_grids.append(image_words_grid(img))

print("Example grid shape:", word_grids[0].shape)
print("Example grid IDs:\n", word_grids[0])


  0%|          | 0/300 [00:00<?, ?it/s]


NameError: name 'kmeans' is not defined

In [17]:
def assign_words(X, centers):
    x2 = (X**2).sum(axis=1, keepdims=True)
    c2 = (centers**2).sum(axis=1)[None, :]
    dist = x2 + c2 - 2.0 * (X @ centers.T)
    return dist.argmin(axis=1).astype(np.int32)


In [15]:
# =========================
# 9) Build LOCAL transactions (3x3 neighborhoods)
#    Each transaction is a set of word IDs co-occurring locally.
# =========================
NEIGH = 1  # 1 => 3x3 window; 2 => 5x5

def local_transactions_from_grid(grid, neigh=NEIGH):
    gh, gw = grid.shape
    tx = []
    for r in range(gh):
        for c in range(gw):
            r0, r1 = max(0, r-neigh), min(gh, r+neigh+1)
            c0, c1 = max(0, c-neigh), min(gw, c+neigh+1)
            window = grid[r0:r1, c0:c1].ravel()
            # Use unique items so transaction is a set
            items = list(set(int(x) for x in window))
            # prefix items so TransactionEncoder treats them as categorical tokens
            items = [f"w{it}" for it in items]
            tx.append(items)
    return tx

transactions = []
for grid in tqdm(word_grids):
    transactions.extend(local_transactions_from_grid(grid))

print("Total transactions:", len(transactions))
print("Example transaction:", transactions[0][:10])


NameError: name 'word_grids' is not defined

In [ ]:
def mine_pair_rules_from_sets(transactions):
    item_count = Counter()
    pair_count = Counter()
    n_tx = len(transactions)

    for s in transactions:
        items = sorted(s)
        for a in items:
            item_count[a] += 1
        for i in range(len(items)):
            for j in range(i+1, len(items)):
                pair_count[(items[i], items[j])] += 1

    rules = []
    for (a, b), pab in pair_count.items():
        sup = pab / n_tx
        conf_ab = pab / item_count[a]
        conf_ba = pab / item_count[b]
        rules.append((a, b, sup, conf_ab, conf_ba))

    return rules


In [ ]:
MIN_SUPPORT = 0.005
MIN_CONF = 0.25

sym_pairs = []
for a, b, sup, conf_ab, conf_ba in mine_pair_rules_from_sets(transactions):
    if sup >= MIN_SUPPORT and conf_ab >= MIN_CONF and conf_ba >= MIN_CONF:
        avg_conf = 0.5 * (conf_ab + conf_ba)
        sym_pairs.append((avg_conf, sup, a, b))

sym_pairs.sort(reverse=True)
print("Top symmetric pairs:", sym_pairs[:10])


In [ ]:
def apply_single_merge(transactions, a, b, new_id):
    out = []
    for s in transactions:
        s2 = set(s)
        if a in s2 and b in s2:
            s2.remove(a)
            s2.remove(b)
            s2.add(new_id)
        out.append(s2)
    return out


In [ ]:
# =========================
# 13) (Next step) Wrap merging into an iterative loop (skeleton)
#     This is where you recreate the full paper-style iterative merging.
# =========================
def mine_pair_rules(transactions, min_support=0.005, min_conf=0.25):
    import pandas as pd
    te = TransactionEncoder()
    T = te.fit(transactions).transform(transactions)
    df = pd.DataFrame(T, columns=te.columns_)
    freq = apriori(df, min_support=min_support, use_colnames=True)
    rules = association_rules(freq, metric="confidence", min_threshold=min_conf)
    pair_rules = rules[
        rules["antecedents"].apply(is_singleton) &
        rules["consequents"].apply(is_singleton)
    ].copy()
    return pair_rules

def best_symmetric_pair(pair_rules):
    conf_map = {}
    for _, row in pair_rules.iterrows():
        a = next(iter(row["antecedents"]))
        b = next(iter(row["consequents"]))
        conf_map[(a, b)] = float(row["confidence"])
    best = None
    for (a, b), cab in conf_map.items():
        cba = conf_map.get((b, a))
        if cba is None:
            continue
        avg = 0.5 * (cab + cba)
        if (best is None) or (avg > best[0]):
            best = (avg, a, b)
    return best  # (avg_conf, a, b) or None

def iterative_merging(transactions, iters=5, min_support=0.005, min_conf=0.25):
    tx = transactions
    merges = []
    for t in range(iters):
        pair_rules = mine_pair_rules(tx, min_support=min_support, min_conf=min_conf)
        best = best_symmetric_pair(pair_rules)
        if best is None:
            print(f"Stop: no symmetric pair at iter {t}.")
            break
        avg_conf, a, b = best
        new_token = f"p{t}({a}+{b})"
        tx = apply_single_merge(tx, a, b, new_token)
        merges.append((new_token, a, b, avg_conf))
        print(f"Iter {t}: merge {a}<->{b} avg_conf={avg_conf:.3f} => {new_token}")
    return tx, merges

# Run a tiny number of merges to start
tx_final, merges = iterative_merging(transactions, iters=3, min_support=MIN_SUPPORT, min_conf=MIN_CONF)
print("Merges:", merges)
